# Creacion y prueba de la implementacion del algoritmo de descenso de gradiente

Se debe probara el algoritmo usando un dataset de 3 columnas y 1000 filas para las cuales se aplicara regresion sobre una de las variables generando una funcion de regresion de la forma $f(x) = w_1 x_1 + w_2 x_2 + w_0$.

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [24]:
training_df = pd.DataFrame(columns=['var_a', 'var_b', 'var_c'])
training_df

,var_a,var_b,var_c


La funcion ``` np.random.rand() ``` genera numeros aleatorios entre [0, 1) de una poblacion con distribucion uniforme

In [64]:
n_values = 1000
#Coeficientes que deberia predecir el modelo como vector de pesos
w0 = 0.86
w1 = 1.2
w2 = -0.756
training_df['var_b'] = np.random.rand(n_values)
training_df['var_c'] = np.random.rand(n_values)
training_df['var_a'] = [w0 + training_df['var_b'][i]*w1 + training_df['var_c'][i]*w2 for i in range(len(training_df))]
training_df

,var_a,var_b,var_c
0,1.323503,0.780373,0.625588
1,1.166403,0.438199,0.290260
2,0.759212,0.308122,0.622399
3,1.734317,0.775125,0.073853
4,1.283433,0.366285,0.021308
...,...,...,...
995,1.691826,0.750912,0.091625
996,0.336288,0.116265,0.877290
997,1.386687,0.768072,0.522485
998,0.912278,0.145712,0.162139


# Clase prediction_model

El modelo de predicción es abstraído en una clase que recibe un data set, una variable dependiente que debe ser el nombre de alguna columna en el data set ingresado, y un arreglo de variables independientes en función de las cuales se va a generar el modelo para estimar la variable dependiente. Esta clase tiene únicamente la responsabilidad de calcular los pesos para las variables independientes y no se encarga de, por ejemplo, efectuar el preprocesamiento del dataset ingresado.

In [112]:
class prediction_model:
    def __init__(self, data_set: pd.DataFrame, dep_var: str, ind_vars: list[str], learning_rate: float, treshold: float):
        self.data_set = data_set
        self.dep_var = dep_var
        self.ind_vars = ind_vars
        self.learning_rate = learning_rate
        self.weights: pd.Series[float] = None
        self.treshold = treshold
        self.max_iter = 50000
    
    '''
    Función que retorna el vector con los pesos de las variables independientes en un objeto 
    Series de pandas. El algoritmo solo se ejecuta la primera vez que el metodo es invocado,
    en cada llamada subsecuente al método se retorna el vector de pesos ya calculados anteriormente.
    '''
    def get_model(self, ):
        #Si el modelo ya se calculó se retorna
        if self.weights:
            return self.weights
        else:
            #Se inicializan los pesos en cero
            self.weights = pd.Series([0.0]*(len(self.ind_vars)+1))
            y = self.data_set[self.dep_var]
            x = self.data_set[self.ind_vars]
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=0)
            x_train.insert(0, "ind_term", [1.0]*len(x_train)) #Insertar columna de 1's para el termino independiente w0

            i = 0
            while True:
                y_pred = np.dot(self.weights, x_train.T)
                err = y_train - y_pred
                dw = (2/len(x_train)) * np.dot(err.T, x_train)
                self.weights = self.weights + (self.learning_rate*dw)
                i += 1
                if sum(err) < self.treshold:
                    print("Number of iterations:", i)
                    break
                
            return self.weights
            

In [113]:
example = prediction_model(training_df, 'var_a', ['var_b', 'var_c'], 0.001, 0.001)
w = example.get_model()
w

Number of iterations: 86231


0    0.859976
1    1.200020
2   -0.755976
dtype: float64